In [1]:
#!/usr/bin/python
#coding: utf-8

import re, glob
dirname='/media/DATA/Master/Analyse_donnees_text/data_suj1/train/'
d_entities={}
files=glob.glob(dirname+'*.ent')
for file in files:
    with open(file, 'r') as f_in :
        for line in f_in :
            line = re.sub(r'\n', '', line)
            numId = re.sub(r"(.+)\t([A-Z][a-z]+)\t(.+)", r'\1', line)
            cat = re.sub(r"(.+)\t([A-Z][a-z]+)\t(.+)", r'\2', line)
            nom = re.sub(r"(.+)\t([A-Z][a-z]+)\t([0-9]+-[0-9]+) ([A-Za-z]+)", r'\4', line)
            if re.match(r"[0-9]+", numId) and nom not in d_entities:
                d_entities[nom]=cat
#print(d_entities)

In [3]:
import re, glob
dirname='/media/DATA/Master/Analyse_donnees_text/data_suj1/train/'
d_entities={}
p=re.compile(r'\d+\t([A-Za-z]+)\s\d+-\d+\s([A-Za-z ]+)(\s\d+-\d+\s([A-Za-z ]+))?')

files=glob.glob(dirname+'*.ent')
for file in files:
    with open(file, 'r') as fichier:
        for line in fichier:
            m=p.match(line)
            if not m: break; 
            cat = m.group(1)
            nom = m.group(2)
            if cat =='Bacteria': continue;
            if nom not in d_entities:
                d_entities[nom]=cat
            if m.group(4) is not None and m.group(4) not in d_entities:
                d_entities[m.group(4)]=cat
                
print(d_entities)


{'Gastrointestinal': 'HostPart', 'equipment': 'Environment', 'sugar cane stems': 'HostPart', 'Propella biofilm reactor': 'Environment', 'light harvesting antenna': 'HostPart', 'Guadeloupe': 'Geographical', 'complex media': 'Environment', 'cattle': 'Host', 'Oryza sativa L': 'Host', 'radioactive waste': 'Environment', 'tuberculoid granulomas with caseous lesions': 'HostPart', 'extracellular': 'HostPart', 'Utah': 'Geographical', 'guar gum': 'HostPart', 'insect endosymbiont': 'HostPart', 'woody perennials': 'Host', 'Cuba': 'Geographical', 'British soldiers': 'Host', 'north of France': 'Geographical', 'Euprymna scolopes': 'Host', 'intestinal tract of': 'HostPart', 'United States': 'Geographical', 'Lake Magadi': 'Geographical', 'soil contaminated with PCBs': 'Soil', 'fruit trees': 'Host', 'infected males being converted into functional phenotypic females': 'Host', 'women': 'Host', 'aquatic environments': 'Water', 'cell': 'HostPart', 'seeds': 'HostPart', 'sugar cane': 'Host', 'light organ': '

In [20]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize

#for file in files:
file='/media/DATA/Master/Analyse_donnees_text/data_suj1/dev/BTID-10095.txt'
with open(file, 'r') as fichier:
    text=fichier.read()
    words=word_tokenize(text, 'english')
    tagged=nltk.pos_tag(words)
    #print(tagged)
    compteur = 0
    for mot in tagged:
        #print(mot[0])
        if mot[0] == 'in':
            if tagged[compteur+1][1]== 'NN' and tagged[compteur+2][1]== 'NN':
                print(mot[0] + ' ' +tagged[compteur+1][0] + ' ' + tagged[compteur+2][0])
        compteur += 1
       
    
    

in root nodulation


In [21]:
#import nltk
#nltk.download()